# 作業工程計画
2023.2.21作成<br>

1. 受領データを21夏から22冬までマージ
2.　エラーラベルを付記
3.　参加回数を修正
4.　1-3までをエクセルで出力

# ライブラリ

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
import re 

## 村上さん pandas_tool

In [2]:
"""
村上さんtoolbox
"""
# pandas 基礎集計クラス
import numpy as np
import pandas as pd
import copy
import seaborn as sns
from itertools import combinations

#http://qiita.com/tanemaki/items/2ed05e258ef4c9e6caac

# Jupyterで表示するためには、最初に以下を実行すること
%matplotlib inline

# Static Classで設計する
class pandas_tool:
    
    # All in one チェック（Jupyterのみ）
    def all_basic_summary_jupyter(df):
        print("■ 型の確認")
        display(pandas_tool.type(df))
        print("■ 数値型の確認")
        display(pandas_tool.summary(df)[0])
        print("■ カテゴリ型の確認")
        cate_var_data = list(df.select_dtypes(include=['object']).columns)
        ret = pandas_tool.freq(df,cate_var_data)
        for d in ret:
            display(pd.DataFrame(d))
            print("---------------")
        print("■ 欠損の確認")
        display(pandas_tool.check_missing(df))
    
    # 相関関係可視化（Jupyterのみ）
    def all_value_relation_visualize(df):
        #sns.set_context("poster", 1.2, {"lines.linewidth": 3})
        sns.pairplot(df,size=5)
    
    # カテゴリ変数でのヒートマップ（Jupyterのみ）
    def make_heatmap(df,x,y,value):
        target_df = df.pivot_table(index=x,values=value,columns=y)
        sns.heatmap(target_df, annot=True, fmt='1.1f', cmap='Blues')
    
    # 散布図（Jupyterのみ）
    def make_scatter_chart(df,x,y):
        #sns.jointplot(x=x, y=y, data=df, kind="hex")
        sns.jointplot(x=x, y=y, data=df)
    
    # 組み合わせでヒートマップを作成（Jupyterのみ）
    def all_make_heatmap(df,var_list,value):
        col_num = 2
        var_list_set = list(combinations(var_list,2))
        
        fig, axes = plt.subplots(int(len(var_list_set)/col_num)+1, col_num, figsize=(18,3+6.5*int(len(var_list_set)/col_num)))
        
        for i,target in enumerate(var_list_set):
            target_df = df.pivot_table(index=target[0],values=value,columns=target[1])
            sns.heatmap(target_df, annot=True, fmt='1.1f', cmap='Blues', ax=axes[int(i/col_num), i%col_num])
            
        plt.tight_layout()
    
    # 数値集計
    def summary(df,view=False):
        ret=df.describe()
        mis_ret=df.isnull().sum()
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print("・統計量")
            print(ret)
            print("・欠損値")
            print(mis_ret)
            pd.set_option("display.max_columns",param)
        return ret,mis_ret
    
    # 型チェック
    def type(df,view=False):
        ret = df.dtypes
        if view:
            param=pd.get_option("display.max_rows")
            pd.set_option("display.max_rows",1000)
            print(ret)
            pd.set_option("display.max_rows",param)
        return ret
    
    # 欠損チェック
    def check_missing(df,view=False):
        not_null_df=df.notnull()
        ret=pd.DataFrame()
        for name in not_null_df.columns:
            tmp_df=not_null_df[name].value_counts()
            tmp_df.name=name
            ret = pd.concat([ret,tmp_df],axis=1)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 欠損値のオブザベーションを抽出
    def get_miss_data(df,column,view=False):
        ret=df[df[column].isnull()]
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        return ret
    
    # 欠損値を中央値で補完
    def fill_miss_med(df,var_name):
        var=df[var_name].median()
        df[var_name].fillna(var,inplace=True)
        return df
    
    # 欠損値を0で補完
    def fill_miss_zero(df,var_name):
        df[var_name].fillna(0,inplace=True)
        return df
    
    # 特定の値を欠損とみなす
    def apply_miss_value(df,var_name,value):
        df[var_name]=df[var_name].replace(value,np.nan)
        return df
    
    # 重複チェック
    def check_dup(df,columns,view=False):
        ret=pd.DataFrame()
        for name in columns:
            dup_cnt=df[name].duplicated().sum()
            tmp_df = pd.DataFrame({'var_name':[name],'dup_cnt':[dup_cnt]})
            ret = pd.concat([ret,tmp_df],axis=0,ignore_index= True)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 組み合わせ重複チェック
    def check_dup_comb(df,columns,view=False):
        ret = df[columns].duplicated().sum()
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # ユニークデータ取得
    def get_uniq_data(df,uniq_key,sort_key,keep='first'):
        ret = df.sort_values(by=sort_key)
        ret.drop_duplicates(subset=uniq_key, keep=keep, inplace=True)
        return ret
    
    # カテゴリ集計
    def freq(df,columns,view=False):
        ret=list()
        for name in columns:
            tmp_df=df[name].value_counts()
            tmp_df.name=name
            #ret = pd.concat([ret,tmp_df],axis=1)
            ret.append(tmp_df)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            for r in ret:
                print(r)
                #display(r)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 複雑な集計
    def tabulate(df,row,col=None,var='',func=np.sum,view=False):
        if var == '':
            tmp_df=df.reset_index(drop=False,inplace=False)
            ret=pd.pivot_table(data=tmp_df, values='index', index=row, columns=col, aggfunc='count', dropna=False, fill_value=0 ,margins = False)
            tmp_df=None
        else:
            ret=pd.pivot_table(data=df, values=var, index=row, columns=col, aggfunc=func, dropna=False, fill_value=0 ,margins = False)
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # マージ
    def merge(df1,df2,key,how,view=True):
        if view:
            print("df1のキー重複")
            pandas_tool.check_dup_comb(df1,key,True)
            print("df2のキー重複")
            pandas_tool.check_dup_comb(df2,key,True)
            
            print("df1のオブザベーション:{0}".format(len(df1)))
            print("df2のオブザベーション:{0}".format(len(df2)))
        
        ret=pd.merge(df1,df2,how=how,on=key)
        
        if view:
            print("mergeのオブザベーション:{0}".format(len(ret)))
        
        return ret
    
    # Rank
    def rank(df,var,num,suffix='_rank',check=False):
        labels=[i for i in range(0,num)]
        df[var+suffix]=pd.qcut(df[var], num, labels=labels)
        
        # check data
        if check:
            ret=pd.DataFrame()
            max_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func=np.max,view=False)
            max_df.name='max'
            min_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func=np.min,view=False)
            min_df.name='min'
            cnt_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func='count',view=False)
            cnt_df.name='count'
            ret=pd.concat([ret,min_df,max_df,cnt_df],axis=1)
            return df,ret
            
        return df
    
    # Rank適用(min基準)
    def apply_rank(df,rank_df):
        tmp_df=copy.deepcopy(rank_df)
        tmp_df.reset_index(drop=False,inplace=True)
        target_name=tmp_df.columns[3]
        tmp_df.columns=["rank","min","max","cnt"]
        
        def judge_thld(row):
            ret_var = -1
            cond_list = ["if 0 : ret_var = 0"]
            
            for i in range(1,len(tmp_df)):
                cond_list.append("elif row < " +str(tmp_df.ix[i,'min'])+ " : ret_var = " + str(tmp_df.ix[i-1,'rank']))
            
            cond_list.append("else: ret_var = " + str(tmp_df.ix[len(tmp_df)-1,'rank']))
            cond_str="\r\n".join(cond_list)
            # ローカル辞書をexecと共有する
            local_dict=locals()
            exec(cond_str,local_dict)
            return local_dict["ret_var"]
        
        df[target_name+"_rank"]=df[target_name].apply(judge_thld)
        return df
    
    # Min%以下はMin%点に、Max%以上はMax%点にクリップする
    def clip_min_max(df,col_list,apply_df=None,max_pct=0.99,min_pct=0.01):
        p_min = df[col_list].quantile(min_pct)
        p_max = df[col_list].quantile(max_pct)
        
        df[col] = df[col_list].clip(p_min,p_max,axis=1)
        
        # もしも適用先のデータがあるならば（例えば検証データ）対応
        if apply_df is not None:
            apply_df[col] = apply_df[col_list].clip(p_min,p_max,axis=1)
            return df,apply_df
        else:
            return df
    
    
    # 文字列→数値変換
    def conv_float(df,column,percent_flg=False):
        
        def conv_f(row):
            if row[column] == "" or row[column] is np.nan:
                return np.nan
            else:
                return float(row[column])
        
        df[column]=df[column].str.replace("\\","").str.replace(",","").str.replace("%","").str.strip()
        df[column]=df.apply(conv_f,axis=1)
        
        if percent_flg:
            df[column]=df[column]/100
        
        return df

# データの結合

## 読み込み用関数

In [20]:
#3始まりスタッフコードを、1始まりスタッフコードに変換
def change_3_to_1(row_input):
    code = row_input['コード']
    staffcode = int("1" + code[1:])
    return staffcode

In [31]:
#予測データの設問情報の分野名を分野マスタで修正
def merge_df(df1, df2, merge_key):
    df = pandas_tool.merge(df1,df2, key = [merge_key],how = 'left')
    return df

## 過去データ(21~22夏)

In [60]:
#過去データ読み込み 

path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#データが置いてあるフォルダパス

# csvファイルの読み込みと空のリストに追加
name_csv = "採点者情報0720+0818_pretreated_v2_jpn.csv" #ファイル名
path_file = r'{p}/{n}'.format(p = path_folder, n = name_csv)#ファイルパスとファイル名
df_old_jpn = pd.read_csv(filepath_or_buffer = path_file, sep=",")

# csvファイルの読み込みと空のリストに追加
name_csv = "採点者情報0720+0818_pretreated_v2_math.csv" #ファイル名
path_file = r'{p}/{n}'.format(p = path_folder, n = name_csv)#ファイルパスとファイル名
df_old_math = pd.read_csv(filepath_or_buffer = path_file, sep=",")

# csvファイルの読み込みと空のリストに追加
name_csv = "採点者情報0720+0818_pretreated_v2_eng.csv" #ファイル名
path_file = r'{p}/{n}'.format(p = path_folder, n = name_csv)#ファイルパスとファイル名
df_old_eng = pd.read_csv(filepath_or_buffer = path_file, sep=",")

/Users/s.ogura/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (13,18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [61]:
#設問情報を加えない、全科目の結合df
collist=['スタッフコード', '年度', '採点回','学年', '科目','分野','割当', '身分', '年齢','大学',
       '学部', '登録試験点数', '参加回数', '等級',  '完全一致率','偏差値']
df1=df_old_jpn[collist]
df2=df_old_math[collist]
df3=df_old_eng[collist]
df_old =[]
df_old = df1.append(df2)
df_old = df_old.append(df3)

#等級を除外するなどの処理を行う関数を実行
def rem_tokyu(df_input):
    df = df_input.copy()
    #等級を選別する。
    colname='等級'
    df[colname] = df[colname].astype(str)
    df = df[df[colname].isin(["1","2",'01','02'])]
    #等級の列を除外する
    df = df.drop(columns = colname)
    return df
df_old = rem_tokyu(df_old)

In [62]:
#過去データの偏差値更新のため、偏差値のシートを読み込む

"""
1. 予測データを読み込む
2.　必要な列名を統一する
"""
# 
path_name = "/Users/s.ogura/Documents/CRLEA/data/rawdata/CRLEA採点者情報入力様式_v1.1_大学追加_冬.xlsx"
path_file =  r'{}'.format(path_name)
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names
# ['2212偏差値最新化', 'データ入力様式_採点者', 'Sheet2', 'データ入力様式(記入例)']
df = []
df = pd.read_excel(path_file, sn_list[0], header=0, index_col=None,skiprows=None,usecols=['大学', '学部', '偏差値'])#headerを修正。sn_listが異なることもある。
df = df.rename(columns = {'過去参加回数':'参加回数'})
df_hensa = df.copy()

In [66]:
#分野情報を割り当て、採点回、年度をキーにして結合する
df1 = df_old
df2 = df_hensa
df = pandas_tool.merge(df1,df2, key = ['大学', '学部'],how = 'left')
df['偏差値_y'] = df['偏差値_y'].fillna(df['偏差値_x'], inplace=False)
df = df.rename(columns = {'偏差値_y':'偏差値'})
colname='偏差値'
df[colname] = df[colname].replace({'-':np.nan,
                                    '空白':np.nan,
                                    '短大':np.nan,
                                    '不明':np.nan,
                                    "大学不明":np.nan,
                                    '専門':np.nan,
                                    'なし':np.nan,
                                    '該当なし':np.nan})
df[colname] = df[colname].astype(float)
df = df.drop(['偏差値_x'], axis=1)
collist=['スタッフコード', '年度', '採点回', '学年', '科目', '分野', '割当', '身分', '年齢', '大学', '学部',
       '偏差値', '登録試験点数', '参加回数', '完全一致率']
df = df[collist]
df_2122summer = df.copy()

df1のキー重複
20317
df2のキー重複
104
df1のオブザベーション:22296
df2のオブザベーション:2953
mergeのオブザベーション:22738


## 22秋前半

In [51]:
#22秋前半を読み込み
"""
path_name = ファイルパス
sn_list = エクセルシート名のリスト
"""
path_name = "/Users/s.ogura/Documents/CRLEA/data/intermediatedata/CRLEA様採点者情報入力様式_v1.1_大学追加_0929DAcheck_分野結合済.xlsx"#ここを編集
path_file =  r'{}'.format(path_name)
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names

# Excelファイルの読み込み
df = pd.read_excel(path_file, sn_list[0], header=0, index_col=None,skiprows=None,#sn_listもたまに順番が異なっている。確認する。
                       dtype = {'割当': object, '年齢':int})#,usecols = [0,1,2,3,4]
df = df.drop(['完全一致率'], axis=1)
df = df.rename(columns = {'過去参加回数':'参加回数'})
df_22fall_first_index = df.copy()

In [52]:
#22秋前半の一致率を読み込み
"""
path_name = ファイルパス
sn_list = エクセルシート名のリスト
"""
path_name = "/Users/s.ogura/Documents/CRLEA/data/rawdata/【22秋】配置割当単位・モニタリング一致率.xlsx"#ここを編集
path_file =  r'{}'.format(path_name)
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names

# Excelファイルの読み込み
df = pd.read_excel(path_file, sn_list[1], header=0, index_col=None,skiprows=None,usecols = [0,5],dtype = {'コード':str})

df['スタッフコード']= df.apply(change_3_to_1, axis = 1)
df = df.drop(['コード'], axis=1)
df = df.rename(columns={'一致率': '完全一致率'})
df_22fall_first_value = df.copy()

In [53]:
df_22fall_first = merge_df(df_22fall_first_index, df_22fall_first_value, 'スタッフコード')

df1のキー重複
0
df2のキー重複
0
df1のオブザベーション:3265
df2のオブザベーション:2657
mergeのオブザベーション:3265


## 22秋後半

In [54]:
#22秋後半を読み込み
"""
path_name = ファイルパス
sn_list = エクセルシート名のリスト
"""
path_name = "/Users/s.ogura/Documents/CRLEA/data/intermediatedata/CRLEA様採点者情報入力様式_v1.1_大学追加_秋後半_分野結合済.xlsx"#ここを編集
path_file =  r'{}'.format(path_name)
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names

# Excelファイルの読み込み
df = pd.read_excel(path_file, sn_list[0], header=0, index_col=None,skiprows=None,#sn_listもたまに順番が異なっている。確認する。
                       dtype = {'割当': object, '年齢':int})#,usecols = [0,1,2,3,4]
df = df.drop_duplicates(subset='スタッフコード')
df = df.drop(['完全一致率'], axis=1)
df = df.rename(columns = {'過去参加回数':'参加回数'})
df_22fall_latter_index = df.copy()

In [55]:
#22秋後半の一致率を読み込み
"""
path_name = ファイルパス
sn_list = エクセルシート名のリスト
"""
path_name = "/Users/s.ogura/Documents/CRLEA/data/rawdata/【22秋】配置割当単位・モニタリング一致率.xlsx"#ここを編集
path_file =  r'{}'.format(path_name)
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names

# Excelファイルの読み込み
df = pd.read_excel(path_file, sn_list[0], header=0, index_col=None,skiprows=None,usecols = [0,5],dtype = {'コード':str})

df['スタッフコード']= df.apply(change_3_to_1, axis = 1)
df = df.drop(['コード'], axis=1)
df = df.rename(columns={'一致率': '完全一致率'})
df_22fall_latter_value = df.copy()

In [56]:
df_22fall_latter = merge_df(df_22fall_latter_index, df_22fall_latter_value, 'スタッフコード')

df1のキー重複
0
df2のキー重複
0
df1のオブザベーション:3376
df2のオブザベーション:2656
mergeのオブザベーション:3376


## 22冬

In [44]:
#22冬を読み込み
"""
path_name = ファイルパス
sn_list = エクセルシート名のリスト
"""
path_name = "/Users/s.ogura/Documents/CRLEA/data/intermediatedata/CRLEA採点者情報入力様式_v1.1_大学追加_冬_分野結合済.xlsx"#ここを編集
path_file =  r'{}'.format(path_name)
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names

# Excelファイルの読み込み
df = pd.read_excel(path_file, sn_list[0], header=0, index_col=None,skiprows=None,#sn_listもたまに順番が異なっている。確認する。
                       dtype = {'割当': object, '年齢':int})#,usecols = [0,1,2,3,4]
df = df.drop_duplicates(subset='スタッフコード')
df = df.drop(['完全一致率'], axis=1)
df_22winter_index = df.copy()

In [45]:
#22冬の一致率を読み込み
"""
path_name = ファイルパス
sn_list = エクセルシート名のリスト
"""
path_name = "/Users/s.ogura/Documents/CRLEA/data/rawdata/【22冬】配置割当単位・モニタリング一致率.xlsx"#ここを編集
path_file =  r'{}'.format(path_name)
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names

# Excelファイルの読み込み
df = pd.read_excel(path_file, sn_list[0], header=0, index_col=None,skiprows=None,usecols = [0,5],dtype = {'コード':str})

df['スタッフコード']= df.apply(change_3_to_1, axis = 1)
df = df.drop(['コード'], axis=1)
df = df.rename(columns={'一致率': '完全一致率'})
df_22winter_value = df.copy()

In [46]:
df_22winter = merge_df(df_22winter_index, df_22winter_value, 'スタッフコード')

df1のキー重複
0
df2のキー重複
0
df1のオブザベーション:3887
df2のオブザベーション:3076
mergeのオブザベーション:3887


## dfの結合

In [67]:
df_2122summer.columns.unique()

Index(['スタッフコード', '年度', '採点回', '学年', '科目', '分野', '割当', '身分', '年齢', '大学', '学部',
       '偏差値', '登録試験点数', '参加回数', '完全一致率'],
      dtype='object')

In [57]:
df_22fall_first.columns.unique()

Index(['スタッフコード', '年度', '採点回', '学年', '科目', '分野', '割当', '身分', '年齢', '大学', '学部',
       '偏差値', '登録試験点数', '参加回数', '完全一致率'],
      dtype='object')

In [58]:
df_22fall_latter.columns.unique()

Index(['スタッフコード', '年度', '採点回', '学年', '科目', '分野', '割当', '身分', '年齢', '大学', '学部',
       '偏差値', '登録試験点数', '参加回数', '完全一致率'],
      dtype='object')

In [59]:
df_22winter.columns.unique()

Index(['スタッフコード', '年度', '採点回', '学年', '科目', '分野', '割当', '身分', '年齢', '大学', '学部',
       '偏差値', '登録試験点数', '参加回数', '完全一致率'],
      dtype='object')

In [82]:
def unit_dfs(df1, df2, df3, df4):
    df_all =[]
    df_all = df1.append(df2)
    df_all = df_all.append(df3)
    df_all = df_all.append(df4)
    df_all = df_all.drop_duplicates()
    return df_all

In [83]:
df_all = unit_dfs(df_2122summer, df_22fall_first, df_22fall_latter, df_22winter)

In [3]:
#予測データを読み込み
"""
path_name = ファイルパス
sn_list = エクセルシート名のリスト
"""
path_name = "/Users/s.ogura/Documents/CRLEA/data/rawdata/CRLEA採点者情報入力様式_v1.1_大学追加_冬.xlsx"#ここを編集
path_file =  r'{}'.format(path_name)
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names
#sn_list ['2212偏差値最新化', 'データ入力様式_採点者', 'Sheet2', 'データ入力様式(記入例)']

# Excelファイルの読み込み
df = pd.read_excel(path_file, sn_list[1], header=0, index_col=None,skiprows=None,#sn_listもたまに順番が異なっている。確認する。
                       dtype = {'割当': object, '年齢':int})#,usecols = [0,1,2,3,4]

df['登録試験点数'] = df['登録試験点数'].fillna(0)
df['登録試験点数'] = df['登録試験点数'].astype(int)
df['割当'] = df['割当'].replace(10,'10')
df = df.drop(['分野','最終ペース'], axis=1)
df_new_raw = df.copy()

# データ分類ラベルの作成

#科目コードの作成
def make_kamokucode(df_input):
    df = df_input.copy()
    map_dictionary ={"国語" : 10, "数学" :20, "英語" : 30} 
    df['科目コード'] = df['科目'].map(map_dictionary) 
    return df

#train,test,predのデータ分類ラベル
def make_datalabel(df_input):
    df = df_input.copy()
    #x = "偏差値"
    conditions = [
        (df['年度'] == 2022)&(df['採点回'] == '夏'),
        (df['年度'] == 2022)&(df['採点回'] == '冬')]#予測用。修正箇所
    choices = [1,2]
    df.loc[:,'data'] = np.select(conditions, choices, default = 0)
    return df

In [72]:
#naを判定してフラグをつくる関数
def find_error(df_input):
    df = df_input.copy()

    #例外をnanに変換
    colname='偏差値'
    df[colname] = df[colname].replace({'-':np.nan,
                                    '空白':np.nan,
                                    '短大':np.nan,
                                    '不明':np.nan,
                                    "大学不明":np.nan,
                                    '専門':np.nan,
                                    'なし':np.nan,
                                    '該当なし':np.nan})
    df[colname] = df[colname].fillna(0)
    df[colname] = df[colname].astype(float)
   
    colname='登録試験点数'
    df[colname] = df[colname].replace({999:np.nan,
                                    0:np.nan})
    df[colname] = df[colname].fillna(0)
    df[colname] = df[colname].astype(float)
    

    #完全一致率
    colname='完全一致率'
    df[colname] = df[colname].replace({'-':np.nan,
                                        'NG':np.nan,
                                        '不参加':np.nan})
    df[colname] = df[colname].fillna(0)
    df[colname] = df[colname].astype(float)
    
    colname='分野'
    df[colname] = df[colname].fillna(0)
    
    
    #例外を1と判定
    w = '分野'
    x = "登録試験点数"
    y = "偏差値"
    z = "完全一致率"
    conditions = [
        (df[w] == 0),
        (df[x] == 0),
        (df[y] == 0),
        (df[z] == 0)
         ]
    choices = [1, 1, 1, 1]
    df.loc[:,'Error'] = np.select(conditions, choices, default = 0)
    return df

In [87]:
#欠損を含む行にフラグをつける処理を実行
df_all_pret = find_error(df_all)

In [88]:
df_all_pret.columns.values

array(['スタッフコード', '年度', '採点回', '学年', '科目', '分野', '割当', '身分', '年齢', '大学',
       '学部', '偏差値', '登録試験点数', '参加回数', '完全一致率', 'Error'], dtype=object)

### データ数の確認

In [89]:
#全てのデータ数、欠損を含むデータ数、　新データ
df = df_all_pret
data_num_new=[len(df),len(df[df['Error']==1]),len(df[df['Error']==0])]
data_num_new

[32824, 12110, 20714]

## 参加回数の修正

In [91]:
df_all_pret.columns.values

array(['スタッフコード', '年度', '採点回', '学年', '科目', '分野', '割当', '身分', '年齢', '大学',
       '学部', '偏差値', '登録試験点数', '参加回数', '完全一致率', 'Error'], dtype=object)

In [94]:
def revise_sankakaisu(df_input):
    """
    同一採点者を取得
    map関数で、年度、採点回、学年を基準にして番号を振る
    例：
    2022、秋、3年　202233
    2022、秋、2年 202232
    番号順に降順で並びかえる
    参加回数が記入されている、最も古いレコードの参加回数を取得
    それ以外のレコードに番号をふる
    """
    df = df_input.copy()
    df = df.dropna(subset=['参加回数'])
    #if np.isnan(df['学年']):
    df['学年'] = df['学年'].fillna(0)
    collist=['スタッフコード', '年度', '採点回', '学年', '科目', '分野', '割当', '身分', '年齢', '大学',
           '学部', '偏差値', '登録試験点数', '参加回数', '参加回数_修正', '完全一致率', 'Error']
    temp=[]
    s = list(sorted(df['スタッフコード'].unique().astype(int)))#スタッフコード単位で繰り返す
    for i in s:
        df1 = df[df['スタッフコード']==i]
        x = "採点回"
        y = "学年"
        conditions = [
            (df1[x] == '春'),
            (df1[x] == '夏'),
            (df1[x] == '秋'),
            (df1[x] == '冬')]
        choices = ['1','2','3','4']
        df1.loc[:,'採点回_no'] = np.select(conditions, choices, default = 0)
        conditions = [
            (df1[y] == 1),
            (df1[y] == 2),
            (df1[y] == 3)]
        choices = ['2','2','1']#秋は前半後半があるため
        df1.loc[:,'学年_no'] = np.select(conditions, choices, default = 0)
        df1 = df1.astype({'年度': 'str', '学年': 'str'})
        df1['No'] = df1['年度'] + df1['採点回_no'] + df1['学年_no']
        df1 = df1.astype({'No': 'float'})
        df1 = df1.sort_values('No', ascending=True)#番号を使って古いものを上に並びかえる
        df1['No']= range(0, len(df1))#同一スタッフコードの人の、完全一致率をもつレコードに連番をふる
        temp_no = df1.iat[0, 13]#同一スタッフコードの一番古い値を代入。13は参加回数の列
        df1['参加回数_修正']= temp_no + df1['No']
        df1=df1[collist]
        df1 = df1.rename(columns = {'参加回数':'参加回数_修正前',
                               '参加回数_修正':'参加回数'})
        temp.append(df1)
    dfs = temp[0] #集計結果を一つのDFに結合
    for i in temp[1:]:
        dfs = dfs.append(i)
    return dfs

In [95]:
df_revised = revise_sankakaisu(df_all_pret)

/Users/s.ogura/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [96]:
len(df_revised)

31491

In [100]:
#修正されているか確認している
df = df_revised
df = df[df['スタッフコード']==1070047256]
df.head(10)

,スタッフコード,年度,採点回,学年,科目,分野,割当,身分,年齢,大学,学部,偏差値,登録試験点数,参加回数_修正前,参加回数,完全一致率,Error
750,1070047256,2021,夏,2.0,国語,評論,01,社会人,46.0,獨協大学,外国語学部,61.3,80.0,4.0,4.0,0.897959,0
1886,1070047256,2021,秋,3.0,国語,評論,12,社会人,46.0,獨協大学,外国語学部,61.3,80.0,5.0,5.0,0.714286,0
3306,1070047256,2021,秋,2.0,国語,小説,04,社会人,46.0,獨協大学,外国語学部,61.3,80.0,6.0,6.0,0.827160,0
4693,1070047256,2021,冬,2.0,国語,小説,04,社会人,46.0,獨協大学,外国語学部,61.3,80.0,7.0,7.0,0.810000,0
6154,1070047256,2022,夏,2.0,国語,小説,03,社会人,46.0,獨協大学,外国語学部,61.3,80.0,4.0,8.0,0.775000,0
219,1070047256,2022,秋,3.0,国語,評論,02,社会人,47.0,獨協大学,外国語学部,61.3,80.0,5.0,9.0,0.803922,0
278,1070047256,2022,秋,1.0,国語,評論,02,社会人,47.0,獨協大学,外国語学部,61.3,80.0,5.0,10.0,0.835616,0
673,1070047256,2022,冬,1.0,国語,評論,02,社会人,47.0,獨協大学,外国語学部,61.3,80.0,7.0,11.0,0.919753,0


# データマートの出力

In [101]:
#分野結合済データを出力する。
df1 = df_all
df2 = df_all_pret
df3 = df_revised

name_file = "採点者品質予測_21夏から22冬まで_結合データ" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名

with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name_file)) as writer:
    df1.to_excel(writer, sheet_name='結合データ',encoding='utf-8-sig', index = False)
    df2.to_excel(writer, sheet_name='エラーラベル付きデータ',encoding='utf-8-sig', index = False)
    df3.to_excel(writer, sheet_name='参加回数修正データ',encoding='utf-8-sig', index = False)

# 重複チェック(ここまで実行する)

In [102]:
#欠損を含む行数 過去データ
df = df_revised
data_dup_check=[len(df),len(df[df.duplicated(subset=['スタッフコード', '参加回数'],keep=False)])]
data_dup_check

[31491, 0]

In [72]:
"""
Excelの書き出し 
"""
df = df_dup

name = "採点者品質予測_22秋後半納品_受領データ_参加回数異常レコード"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/rawdata"#フォルダパス

#Excel
with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name)) as writer:
    df.to_excel(writer, sheet_name='data1',encoding='utf-8-sig', index = False)

# 重複削除

In [303]:
#train,test,predのデータ分類ラベル
def rem_dup(df_input):
    df = df_input.copy()
    x = "参加回数"
    conditions = [
        (df['年度'] == 2022)&(df['採点回'] == '秋'),
        (df['年度'] == 2022)&(df['採点回'] == '夏'),
        (df['年度'] == 2021)&(df['採点回'] == '冬'),
        (df['年度'] == 2021)&(df['採点回'] == '秋')&(df['学年'] == 3),
        (df['年度'] == 2021)&(df['採点回'] == '秋')&(df['学年'] != 3),
        (df['年度'] == 2021)&(df['採点回'] == '夏')]
    choices = [1,2,3,4,5,6]
    df.loc[:,'時系列順'] = np.select(conditions, choices, default = 0)
    df = df.sort_values('時系列順', ascending=True)
    df = df.drop_duplicates(subset=['スタッフコード', '参加回数'],keep='last')
    return df

In [304]:
df_output = rem_dup(df_comb)

In [305]:
df = df_output
df_dup = df[df.duplicated(subset=['スタッフコード', '参加回数'],keep='last')]

In [307]:
len(df_output)

24269

In [308]:
len(df_dup)

0

In [309]:
"""
Excelの書き出し 
"""
df = df_output

name = "採点者品質予測_22秋納品_整形済み学習データ_参加回数異常レコード削除_v1"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス

#Excel
with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name)) as writer:
    df.to_excel(writer, sheet_name='data1',encoding='utf-8-sig', index = False)

# 過去エクセルデータの編集（参考として残してある。実行不要）

## データ②を縦型に変換

In [3]:
"""
1. 受領データ②を読み込む
2.　採点回ごとにデータフレームに入れる
3.　スタッフコードをキーにして一つのデータフレームとしてマージする
"""
# 
path_name = "/Users/s.ogura/Documents/CRLEA/data/rawdata/②採点者情報_採点に関する情報_0720_名前削除.xlsx"
path_file =  r'{}'.format(path_name)
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names

#夏採点を読み込んで、採点回を列に追加する　df_summer

df = []
df = pd.read_excel(path_file, sn_list[0], header=1, index_col=None,skiprows=None,usecols=list(range(0,24,1)))#headerを修正
df.columns=['スタッフコード', 'スタッフコード3', '科目', '登録試験点数', '等級', '参加回数', '学年', '割当', '分野名',
       '採否時登録時間', '1回目採点満点', '1回目採点得点', '完全混在枚数総数', '完全一致枚数', '完全一致率',
       '範囲内混在枚数総数', '範囲内一致枚数', '範囲内一致率', '最終シール', 'ACTB', 'ACTB理由', 'ACTB登録日',
       '依頼枚数', '枠更新率']
df["採点回"]= "夏"
df_summer = df

#秋採点を読み込んで、採点回を列に追加する　df_fall

df = []
df = pd.read_excel(path_file, sn_list[0], header=1, index_col=None,skiprows=None,usecols=[0,1,2,3] + list(range(24,44,1)))#headerを修正
df.columns=['スタッフコード', 'スタッフコード3', '科目', '登録試験点数', '等級', '参加回数', '学年', '割当', '分野名',
       '採否時登録時間', '1回目採点満点', '1回目採点得点', '完全混在枚数総数', '完全一致枚数', '完全一致率',
       '範囲内混在枚数総数', '範囲内一致枚数', '範囲内一致率', '最終シール', 'ACTB', 'ACTB理由', 'ACTB登録日',
       '依頼枚数', '枠更新率']
df["採点回"]= "秋"
df_fall_f = df

df = []
df = pd.read_excel(path_file, sn_list[0], header=1, index_col=None,skiprows=None,usecols=[0,1,2,3] + list(range(44,64,1)))#headerを修正
df.columns=['スタッフコード', 'スタッフコード3', '科目', '登録試験点数', '等級', '参加回数', '学年', '割当', '分野名',
       '採否時登録時間', '1回目採点満点', '1回目採点得点', '完全混在枚数総数', '完全一致枚数', '完全一致率',
       '範囲内混在枚数総数', '範囲内一致枚数', '範囲内一致率', '最終シール', 'ACTB', 'ACTB理由', 'ACTB登録日',
       '依頼枚数', '枠更新率']
df["採点回"]= "秋"
df_fall_l = df

#冬採点を読み込んで、採点回を列に追加する　df_winter
df = []
df = pd.read_excel(path_file, sn_list[0], header=1, index_col=None,skiprows=None,usecols=[0,1,2,3] + list(range(64,84,1)))#headerを修正
df.columns=['スタッフコード', 'スタッフコード3', '科目', '登録試験点数', '等級', '参加回数', '学年', '割当', '分野名',
       '採否時登録時間', '1回目採点満点', '1回目採点得点', '完全混在枚数総数', '完全一致枚数', '完全一致率',
       '範囲内混在枚数総数', '範囲内一致枚数', '範囲内一致率', '最終シール', 'ACTB', 'ACTB理由', 'ACTB登録日',
       '依頼枚数', '枠更新率']
df["採点回"]= "冬"
df_winter = df

#dataframeの結合
df =df_summer.append(df_fall_f)
df =df.append(df_fall_l)
df = df.append(df_winter)
df = df.rename(columns={'科目':'科目コード'})
df["年度"]= 21
df_2 = df

In [4]:
df_2['参加回数'].unique()

array([ 5.,  3., 14.,  9.,  7., 10.,  8.,  4., 13.,  2.,  6., 11., 16.,
        1., 12., 32., 20., 15., 17., 19., 22., nan, 33., 18., 23., 21.,
       30., 36., 35., 27., 31., 37., 24., 26., 29., 25.,  0., 34., 28.])

In [5]:
len(df_2)

40140

In [5]:
"""
Excelの書き出し 
"""

df = df_2

name = "②採点者情報_採点に関する情報0720_名前削除_縦型v2"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス

#csv
df.to_csv('{}/{}.csv'.format(path_folder,name),encoding='utf-8-sig',index=False)
#Excel
with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name)) as writer:
    df.to_excel(writer, sheet_name='data1',encoding='utf-8-sig', index = False)

## データ④

In [6]:
"""
1. 受領データ④を読み込む
2.　スタッフコードをキーにして一つのデータフレームとしてマージする
"""
# 
path_name = "/Users/s.ogura/Documents/CRLEA/data/rawdata/④採用者情報_個人に関わる詳細_0712.xlsx"
path_file =  r'{}'.format(path_name)
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names

#夏採点を読み込んで、採点回を列に追加する　df_summer

df = []
df = pd.read_excel(path_file, sn_list[0], header=0, index_col=None,skiprows=None)#headerを修正
df = df.rename(columns = {'学生/社会人':'身分'})
df_4 = df.loc[:,['スタッフコード', '身分', '性別', '年齢', '大学', '学部', '偏差値']]

In [8]:
df_4.columns.values

array(['スタッフコード', '身分', '性別', '年齢', '大学', '学部', '偏差値'], dtype=object)

In [9]:
df_4.head()

,スタッフコード,身分,性別,年齢,大学,学部,偏差値
0,1050015912,社会人,女性,58,東北大学医療技術短期大学部,NaN,短大
1,1050740209,社会人,女性,35,上智大学,文学部,69.6
2,1050742436,社会人,女性,53,岡山大学,教育学部,59.6
3,1050852853,社会人,女性,35,福岡女学院大学,人文学部,49
4,1051284876,社会人,女性,29,長崎大学,経済学部,56.6


In [7]:
df_24 = pandas_tool.merge(df_2,df_4,key = 'スタッフコード',how ='left' ,view=True)

df1のキー重複
30105
df2のキー重複
0
df1のオブザベーション:40140
df2のオブザベーション:16300
mergeのオブザベーション:40140


In [11]:
len(df_24)

40140

In [12]:
df_24.head(3)

,スタッフコード,スタッフコード3,科目コード,登録試験点数,等級,参加回数,学年,割当,分野,採否時登録時間,...,依頼枚数,枠更新率,採点回,年度,身分,性別,年齢,大学,学部,偏差値
0,1051286194,3051286194,10,0,02,5.0,1年生,01,評論,108.75,...,5982,0.841525,夏,21,社会人,女性,27.0,中京大学,心理学部,60.6
1,1051284876,3051284876,10,0,02,3.0,1年生,01,評論,97,...,8591,0.963683,夏,21,社会人,女性,29.0,長崎大学,経済学部,56.6
2,1060294576,3060294576,30,27,02,14.0,1年生,-,-,43.5,...,-,-,夏,21,社会人,女性,68.0,立教大学,文学部,65.6


## データ③

In [8]:
"""
1. 受領データ③を読み込む
2.　採点者コード（スタッフコード）、採点回、学年をキーにしてdf_2のデータフレームとマージする
"""
path_name = "/Users/s.ogura/Documents/CRLEA/data/rawdata/③【21夏-冬】採点者別ペース_0414更新.xlsx"
path_file =  r'{}'.format(path_name)
input_file = pd.ExcelFile(path_file)
sn_list= input_file.sheet_names
sn_list = sn_list[0:len(sn_list)-2]
#配列ループ
df = [] #空のリスト
for i in range(0,len(sn_list)): 

    # Excelファイルの読み込み
    ret = pd.read_excel(path_file, sn_list[i], index_col=None)
    #ret
    ret["採点回"]= sn_list[i][0]#sn_listのiつ目の要素から1番目の文字
    ret["学年"]=int(sn_list[i][1])#sn_listのiつ目の要素から2番目の文字    
    # Excel sheetsを空のリストの各要素に追加
    df.append(ret)

    
"""
集計したデータを縦に結合する。
df = df1.append(df2)
"""
dfs = df[0] #集計結果を一つのDFに結合
for i in df[1:]:
    dfs = dfs.append(i)

map_dictionary ={1 : "1年生", 2 :"2年生", 3 : "3年生"} 
dfs['学年'] = dfs['学年'].map(map_dictionary) 
dfs = dfs.rename(columns = {'合計 / 合計 / バツウチ済枚数':'バツウチ済枚数',
                         '合計 / 合計 / 採点済枚数':'採点済枚数',
                         '合計 / 合計 / 採点所要時間':'採点所要時間',
                         'ペース':'最終ペース',
                           '採点者コード':'スタッフコード3'})
dfs=dfs[dfs['科目コード'].isin([10, 20,30])]#科目コード10、20、30を抽出
dfs=dfs.drop(['割当'], axis=1)
df_3 = dfs

In [10]:
col_names_list = [
 'スタッフコード',
 'スタッフコード3',
 '年度',
 '採点回',
 '科目コード',
 '身分',
 '性別',
 '年齢',
 '登録試験点数',
 '大学',
 '学部',
 '偏差値',
 '参加回数',
 '等級',
 '学年',
 '割当',
 '分野名',
 '最終ペース',
 '採否時登録時間',
 '1回目採点満点',
 '1回目採点得点',
 '完全混在枚数総数',
 '完全一致枚数',
 '完全一致率',
 '範囲内混在枚数総数',
 '範囲内一致枚数',
 '範囲内一致率',
 '最終シール',
 'ACTB',
 'ACTB理由',
 'ACTB登録日',
 '依頼枚数',
 '枠更新率']

In [11]:
"""
2つのテーブルを結合させる
df_24:②に④と分野マスタを左外部結合
df_3:③
"""
df1 = df_24
df2 = df_3
df_243 = pandas_tool.merge(df1,df2, key = ["スタッフコード3","採点回","学年","科目コード"],how = 'left')
# 重複行を削除するコードを挿入
df_243= df_243.drop_duplicates()
df_243 = df_243.drop(columns=['製品コード', '採点所要時間', '採点済枚数', 'バツウチ済枚数', '実枚数'])
df_243 = df_243.reindex(columns=col_names_list)

df1のキー重複
4599
df2のキー重複
0
df1のオブザベーション:40140
df2のオブザベーション:12376
mergeのオブザベーション:40140


In [12]:
"""
20220902に追記。
21データは参加回数が1始まりとなっていたため、初心者の参加回数が1と数えられていた。
22年夏データは参加回数を0から数えていたため、初心者は0となっている。
そこで21年データの参加回数を-1して22年のデータに数え方を合わせる。
"""
df_243['参加回数'] = df_243['参加回数']-1

In [15]:
"""
Excelの書き出し 
"""

df = df_243

name = "②採点者情報_採点に関する情報0720_名前削除_縦型_結合データv3"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス

#csv
df.to_csv('{}/{}.csv'.format(path_folder,name),encoding='utf-8-sig',index=False)
#Excel
with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name)) as writer:
    df.to_excel(writer, sheet_name='data1',encoding='utf-8-sig', index = False)

## 受領データ②採点者に関する情報_0818_名前削除.xlsxを整形

In [16]:
"""
1. 受領データを読み込む
2. 列を並び替え
"""
# 
path_name = "/Users/s.ogura/Documents/CRLEA/data/rawdata/②採点者に関する情報_0818_名前削除.xlsx"
path_file =  r'{}'.format(path_name)
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names

#夏採点を読み込んで、採点回を列に追加する　df_summer

df = []
df = pd.read_excel(path_file, sn_list[0], header=0, index_col=None,skiprows=[0,1],usecols=list(range(0,31,1)))#headerを修正
df.columns=['スタッフコード', 'スタッフコード3', '科目コード', '身分','性別','年齢','登録試験点数', '大学', '学部', '偏差値', '参加回数', '等級', '学年', '割当', '分野名',
       '採否時登録時間', '1回目採点満点', '1回目採点得点', '完全混在枚数総数', '完全一致枚数', '完全一致率',
       '範囲内混在枚数総数', '範囲内一致枚数', '範囲内一致率', '最終シール', 'ACTB', 'ACTB理由', 'ACTB登録日',
       '依頼枚数', '枠更新率','最終ペース']
df["採点回"]= "夏"
df["年度"]= 22
df = df.reindex(columns=col_names_list)
df_22summer = df

In [17]:
#旧データと新データを結合
df = df_243.append(df_22summer)
map_dictionary ={"1年生":1, "2年生":2, "3年生":3} 
df['学年'] = df['学年'].map(map_dictionary) 

map_dictionary ={21 : 2021, 22 :2022} 
df['年度'] = df['年度'].map(map_dictionary)

df['割当'] = df['割当'].replace(1,'01')
df['割当'] = df['割当'].replace(2,'02')
df['割当'] = df['割当'].replace(3,'03')
df['割当'] = df['割当'].replace(4,'04')
df['割当'] = df['割当'].replace(5,'05')
df['割当'] = df['割当'].replace(6,'06')
df['割当'] = df['割当'].replace(7,'07')
df['割当'] = df['割当'].replace(8,'08')
df['割当'] = df['割当'].replace(9,'09')
df['割当'] = df['割当'].replace(10,'10')
df['割当'] = df['割当'].replace(11,'11')
df['割当'] = df['割当'].replace(12,'12')
df['割当'] = df['割当'].replace(13,'13')

df_comb = df.copy()
len(df_243)+len(df_22summer)==len(df_comb)

True

In [18]:
"""
Excelの書き出し 
"""

df = df_comb

name = "採点者情報0720+0818_分野修正前_0902"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス

#csv
df.to_csv('{}/{}.csv'.format(path_folder,name),encoding='utf-8-sig',index=False)
#Excel
with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name)) as writer:
    df.to_excel(writer, sheet_name='data1',encoding='utf-8-sig', index = False)

## 分野の修正、設問データの結合

In [ ]:
#設問データを読み込んで結合する。
#採点者データは科目毎にデータフレームを分けて、それぞれで結合した後に再結合するのが良い。
#国語と英語に関しては設問データの分野名が正しい。年度と採点回と割当（ロット番号）で結合できるはず
#変数に対して前処理を行なった後、結合する。
#数学に関しては、分野区分マスタが正しい
#
#

In [19]:
##設問データを読み込んで結合する。
"""
path_name = ファイルパス
sn_list = エクセルシート名のリスト
"""
path_name = "/Users/s.ogura/Documents/CRLEA/data/rawdata/①答案に関する情報_0822_小倉割当追記.xlsx"
path_file =  r'{}'.format(path_name)
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names
#sn_list ['データ入力様式_英語', 'データ入力様式_国語', 'データ入力様式_数学', '空欄埋め方', 'データ入力様式(記入例)']

# Excelファイルの読み込み
df_eng_setsumon = pd.read_excel(path_file, sn_list[0], header=0, index_col=None,skiprows=[1])#headerを修正
df_jpn_setsumon = pd.read_excel(path_file, sn_list[1], header=0, index_col=None,skiprows=[1])#headerを修正
df_math_setsumon = pd.read_excel(path_file, sn_list[2], header=0, index_col=None,skiprows=None)#headerを修正

In [20]:
def make_rank(df,x=str):
    # 最終ペースをランク分け
    if x == "最終ペース":
        conditions = [
        (df[x] >= 600),
        (df[x] >= 500),
        (df[x] >= 400),
        (df[x] >= 300),
        (df[x] >= 200),
        (df[x] >= 100),
        (df[x] >= 0)
         ]
        
        choices = ["600~", "500~600", "400~500", "300~400", "200~300", "100~200","0~100"]
        df.loc[:,'最終ランク'] = np.select(conditions, choices, default = 0)
    # 最終ペースをランク分け
    if x == "年度":
        conditions = [
        (df[x] >= 2020),
        (df[x] >= 0)
         ]
        
        choices = [1, 0]
        df.loc[:,'在宅勤務'] = np.select(conditions, choices, default = 0)
    return df

## 国語科目の前処理 

In [30]:
df = df_jpn_setsumon
#グレーアウトした行を削除
df = df[df['採点回']!='春']
df = df[df['ロット番号']!='基準なし']
#len(df) 346->307

#列の欠損値を補完
df['必須条件の有無'] = df['必須条件の有無'].fillna(0)#必須条件の有無の空白を0に
df['配点要素の数'] = df['配点要素の数'].fillna(df['ポイント数'])#配点要素の数の空白をポイント数と同じ数に
#最終ペースでランク分け、年度で在宅勤務のフラグ
#lst=['最終ペース','年度']
#for i in lst:
#    df = make_rank(df,i)
    
#分野の表記揺れを修正
df['分野'] = df['分野'].replace('古文（現代語訳）','古文（現訳）')
df['分野'] = df['分野'].replace('古文（内容説明）','古文（内説）')
df['分野'] = df['分野'].replace('漢文（現代語訳）','漢文（現訳）')
df['分野'] = df['分野'].replace('漢文（内容説明）','漢文（内説）')

df['ロット番号'] = df['ロット番号'].replace(1,'01')
df['ロット番号'] = df['ロット番号'].replace(2,'02')
df['ロット番号'] = df['ロット番号'].replace(3,'03')
df['ロット番号'] = df['ロット番号'].replace(4,'04')
df['ロット番号'] = df['ロット番号'].replace(5,'05')
df['ロット番号'] = df['ロット番号'].replace(6,'06')
df['ロット番号'] = df['ロット番号'].replace(7,'07')
df['ロット番号'] = df['ロット番号'].replace(8,'08')
df['ロット番号'] = df['ロット番号'].replace(9,'09')
df['ロット番号'] = df['ロット番号'].replace(10,'10')
df['ロット番号'] = df['ロット番号'].replace(11,'11')
df['ロット番号'] = df['ロット番号'].replace(12,'12')

#必要な列のみを抽出
df = df[['年度', '採点回','ロット番号', '学年', '科目', '分野', 'ポイント採点', '文字数',
       'ポイント数', '配点要素の数', '配点',  '必須条件の有無','企画ペース', '最終ペース']]
df = df.rename(columns = {'ロット番号':'割当','最終ペース':'最終ペース_設問'})
df_jpn_setsumon_pretreated = df.copy()

In [136]:
df_jpn_setsumon_pretreated['割当'].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16'], dtype=object)

In [140]:
df_comb[df_comb['科目コード']==10]['割当'].unique()

array(['01', '02', '03', '05', '-', '04', '06', '07', '08', '09', '10',
       '11', '12', 'KS', nan, '不参加', 'パート分けナシ'], dtype=object)

In [31]:
"""
2つのテーブルを結合させる
df1:採点者データの国語だけ抽出
df2:設問データの国語だけ抽出
"""
df1 = df_comb[df_comb['科目コード']==10]
df2 = df_jpn_setsumon_pretreated
df_res = pandas_tool.merge(df1,df2, key = ["年度","採点回","学年","割当"],how = 'left')
df_res= df_res.drop_duplicates()
df_comb_jpn = df_res.copy()
df_res.head()

df1のキー重複
10216
df2のキー重複
0
df1のオブザベーション:10337
df2のオブザベーション:332
mergeのオブザベーション:10337


,スタッフコード,スタッフコード3,年度,採点回,科目コード,身分,性別,年齢,登録試験点数,大学,...,科目,分野,ポイント採点,文字数,ポイント数,配点要素の数,配点,必須条件の有無,企画ペース,最終ペース_設問
0,1051286194,3051286194,2021,夏,10,社会人,女性,27.0,0.0,中京大学,...,国語,評論,0.0,30.0,2.0,2.0,4.0,0.0,330.0,273.3
1,1051284876,3051284876,2021,夏,10,社会人,女性,29.0,0.0,長崎大学,...,国語,評論,0.0,30.0,2.0,2.0,4.0,0.0,330.0,273.3
2,1060309944,3060309944,2021,夏,10,社会人,女性,40.0,26.0,慶応義塾大学,...,国語,評論,0.0,30.0,2.0,2.0,4.0,0.0,330.0,273.3
3,1060311559,3060311559,2021,夏,10,社会人,女性,39.0,25.0,川崎医療福祉大学,...,国語,評論,0.0,30.0,2.0,2.0,4.0,0.0,330.0,273.3
4,1060320722,3060320722,2021,夏,10,社会人,女性,53.0,25.0,桐朋学園大学,...,国語,評論,0.0,30.0,2.0,2.0,4.0,0.0,330.0,273.3


In [32]:
"""
Excelの書き出し 
"""

df = df_comb_jpn

name = "採点者情報0720+0818_国語設問"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス

#csv
df.to_csv('{}/{}.csv'.format(path_folder,name),encoding='utf-8-sig',index=False)
#Excel
with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name)) as writer:
    df.to_excel(writer, sheet_name='data1',encoding='utf-8-sig', index = False)

## 英語科目の前処理

In [21]:
def split_columns(df):
    #文字数に数値が入っている場合、そのまま代入
    if str(df).isdigit() == True: #簡単な例から場合分けする。 文字列か、数値か判定
        return float(df)
    elif '+' in str(df):
        x = str(df).split('+')
        return sum(list(map(float, x)))
    elif '-' in str(df):
        x = str(df).split('-')
        return sum(list(map(float, x)))
    elif '‐' in str(df):
        x = str(df).split('‐')
        return sum(list(map(float, x)))
    else:
        return float(df)

In [22]:
df = df_eng_setsumon
#グレーアウトした行を削除
df = df[df['分野'] != '英単語']#英単語を除外
#len(df) 333->321

#列の欠損値を補完
df = df.fillna(0)
#合計列の追加
df['配点合計'] = df['配点'].apply(split_columns)
df['文字数合計'] = df['文字/単語数'].apply(split_columns)
df['ポイント数合計'] = df['ポイント・ランク・パート数'].apply(split_columns)
df['減点要素の数合計'] = df['減点要素の数'].apply(split_columns)

df['ロット番号'] = df['ロット番号'].replace(1,'01')
df['ロット番号'] = df['ロット番号'].replace(2,'02')
df['ロット番号'] = df['ロット番号'].replace(3,'03')
df['ロット番号'] = df['ロット番号'].replace(4,'04')
df['ロット番号'] = df['ロット番号'].replace(5,'05')
df['ロット番号'] = df['ロット番号'].replace(6,'06')
df['ロット番号'] = df['ロット番号'].replace(7,'07')
df['ロット番号'] = df['ロット番号'].replace(8,'08')
df['ロット番号'] = df['ロット番号'].replace(9,'09')
df['ロット番号'] = df['ロット番号'].replace(10,'10')
df['ロット番号'] = df['ロット番号'].replace(11,'11')
df['ロット番号'] = df['ロット番号'].replace(12,'12')
df['ロット番号'] = df['ロット番号'].replace(13,'13')
df['ロット番号'] = df['ロット番号'].replace(14,'14')
df['ロット番号'] = df['ロット番号'].replace(15,'15')
df['ロット番号'] = df['ロット番号'].replace(16,'16')
df['ロット番号'] = df['ロット番号'].replace(17,'17')
df['ロット番号'] = df['ロット番号'].replace(18,'18')


#必要な列のみを抽出
df = df.drop(['小問','文字/単語数', '配点', 'ポイント・ランク・パート数', '減点要素の数',], axis=1)
df = df.rename(columns = {'ロット番号':'割当','最終ペース':'最終ペース_設問'})

df_eng_setsumon_pretreated = df.copy()

In [23]:
"""
2つのテーブルを結合させる
df1:採点者データの英語だけ抽出
df2:設問データの英語だけ抽出
"""
df1 = df_comb[df_comb['科目コード']==30]
df2 = df_eng_setsumon_pretreated
df_res = pandas_tool.merge(df1,df2, key = ["年度","採点回","学年","割当"],how = 'left')
df_res= df_res.drop_duplicates()
df_comb_eng = df_res.copy()
df_res.head()

df1のキー重複
18908
df2のキー重複
10
df1のオブザベーション:19032
df2のオブザベーション:352
mergeのオブザベーション:19032


,スタッフコード,スタッフコード3,年度,採点回,科目コード,身分,性別,年齢,登録試験点数,大学,...,分野,解答言語,ポイント採点,企画ペース,最終ペース_設問,パターン数,配点合計,文字数合計,ポイント数合計,減点要素の数合計
0,1060294576,3060294576,2021,夏,30,社会人,女性,68.0,27.0,立教大学,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1060448528,3060448528,2021,夏,30,社会人,女性,37.0,62.0,関西大学,...,英訳,英語,0.0,150.0,97.9,3.0,5.0,11.6,5.6,0.0
2,1050742436,3050742436,2021,夏,30,社会人,女性,53.0,60.0,岡山大学,...,英訳,英語,0.0,150.0,97.9,3.0,5.0,11.6,5.6,0.0
3,1060451911,3060451911,2021,夏,30,社会人,女性,47.0,48.0,青山学院大学,...,英訳,英語,0.0,150.0,97.9,3.0,5.0,11.6,5.6,0.0
4,1060454359,3060454359,2021,夏,30,社会人,女性,49.0,51.0,神奈川大学,...,英作文,英語,0.0,400.0,462.1,0.0,4.0,10.0,3.0,1.0


In [159]:
df_eng_setsumon_pretreated['割当'].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18'], dtype=object)

In [162]:
df_comb[df_comb['科目コード']==30]['割当'].unique()

array(['-', '05', '01', '02', '03', '06', '04', '07', '08', '09', '10',
       '13', '11', '12', 'KS', nan, '不参加', 'パート分けナシ', '?'], dtype=object)

In [24]:
"""
Excelの書き出し 
"""

df = df_comb_eng

name = "採点者情報0720+0818_英語設問"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス

#csv
df.to_csv('{}/{}.csv'.format(path_folder,name),encoding='utf-8-sig',index=False)
#Excel
with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name)) as writer:
    df.to_excel(writer, sheet_name='data1',encoding='utf-8-sig', index = False)

## 数学科目の前処理

In [25]:
path_name = "/Users/s.ogura/Documents/CRLEA/data/rawdata/分野区分マスタv4.0.xlsx"
path_file =  r'{}'.format(path_name)
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names

#夏採点を読み込んで、採点回を列に追加する　df_summer

bunya = []
bunya = pd.read_excel(path_file, sn_list[0], header=0, index_col=None,skiprows=None)#headerを修正
bunya = bunya.drop(['ポイント採点', '解答言語'], axis=1)
#bunya = bunya.fillna({'解答言語': '日本語'})
bunya.head()

,科目コード,分野名,分野
0,10,漢文（現代語訳）,漢文(現訳)
1,10,漢文（内容説明）,漢文(内説)
2,10,現代文・小説,小説
3,10,現代文・評論,評論
4,10,現代文・評論（理由説明）,評論


In [167]:
df_math_setsumon.columns.values

array(['年度', '採点回', '学年', '科目', 'ロット番号', '分野', '（1）実点・立式点', '（1）カッコ点・立式点',
       '（1）実点・答え点', '（1）カッコ点・答え点', '（1）実点・その他', '（1）カッコ点・その他',
       '（2）実点・立式点', '（2）カッコ点・立式点', '（2）実点・答え点', '（2）カッコ点・答え点',
       '（2）実点・その他', '（2）カッコ点・その他', '（3）実点・立式点', '（3）カッコ点・立式点',
       '（3）実点・答え点', '（3）カッコ点・答え点', '（3）実点・その他', '（3）カッコ点・その他',
       '（1）ポイント数合計', '（2）ポイント数合計', '（3）ポイント数合計', 'ポイント採点', '（1）配点',
       '（2）配点', '（3）配点', '企画ペース', '最終ペース', '模範解答の記述行数（１）', '模範解答の記述行数（２）',
       '模範解答の記述行数（３）', '補足資料の枚数', '分野（詳細）', '他参照有無', 'Unnamed: 39'],
      dtype=object)

In [26]:
df = df_math_setsumon
#グレーアウトした行を削除
df = df[~((df['採点回'] == '春')&(df['年度']==2019))]#2019春を除外
df = df[~((df['採点回'] == '夏')&(df['年度']==2018)&(df['学年']!=1))]#2018夏2-3年を除外
#len(df) 477->422

#列の欠損値を補完
df = df.fillna(0)#必須条件の有無の空白を0に
#補足資料の枚数列の例外を修正
df['補足資料の枚数'] = df['補足資料の枚数'].replace('補足なし',0)
df = df.rename(columns = {'分野':'分野名'})

#分野名を分野マスタで変換
df1 = df
df2 = bunya
df = pandas_tool.merge(df1,df2, key = ["分野名"],how = 'left')
#不要な列を削除
df = df.drop(['科目コード','分野（詳細）', '他参照有無','Unnamed: 39'], axis=1)
df = df.rename(columns = {'ロット番号':'割当','最終ペース':'最終ペース_設問'})
df = df.drop_duplicates()
df_math_setsumon_pretreated = df.copy()

df1のキー重複
420
df2のキー重複
0
df1のオブザベーション:463
df2のオブザベーション:156
mergeのオブザベーション:463


In [27]:
"""
2つのテーブルを結合させる
df1:採点者データの数学だけ抽出
df2:設問データの数学だけ抽出
"""
df = df_comb[df_comb['科目コード']==20].copy()

#割当を修正
#2022 夏1
df['割当'] = df['割当'].replace(34,'03-4')
df['割当'] = df['割当'].replace(35,'03-5')
df['割当'] = df['割当'].replace(36,'03-6')
#2022夏2
df['割当'] = df['割当'].replace(44,'04-4')
df['割当'] = df['割当'].replace(45,'04-5')
df['割当'] = df['割当'].replace(46,'04-6')
df['割当'] = df['割当'].replace(47,'04-7')
df['割当'] = df['割当'].replace(48,'04-8')
df['割当'] = df['割当'].replace(48,'04-8')
df['割当'] = df['割当'].replace(64,'06-4')
df['割当'] = df['割当'].replace(65,'06-5')
df['割当'] = df['割当'].replace(66,'06-6')

#2022夏3
df['割当'] = df['割当'].replace(24,'02-4')
df['割当'] = df['割当'].replace(25,'02-5')
df['割当'] = df['割当'].replace(83,'08-3')
df['割当'] = df['割当'].replace(84,'08-4')
df['割当'] = df['割当'].replace(138,'13-8')

#2021
df['割当'] = df['割当'].replace('011','01-1')
df['割当'] = df['割当'].replace('012','01-2')
df['割当'] = df['割当'].replace('013','01-3')
df['割当'] = df['割当'].replace('024','02-4')
df['割当'] = df['割当'].replace('025','02-5')

df['割当'] = df['割当'].replace('034','03-4')
df['割当'] = df['割当'].replace('035','03-5')
df['割当'] = df['割当'].replace('036','03-6')

df['割当'] = df['割当'].replace('044','04-4')
df['割当'] = df['割当'].replace('045','04-5')
df['割当'] = df['割当'].replace('046','04-6')
df['割当'] = df['割当'].replace('047','04-7')
df['割当'] = df['割当'].replace('048','04-8')
df['割当'] = df['割当'].replace('048','04-8')
df['割当'] = df['割当'].replace('055','05-5')
df['割当'] = df['割当'].replace('056','05-6')
df['割当'] = df['割当'].replace('057','05-7')
df['割当'] = df['割当'].replace('058','05-8')
df['割当'] = df['割当'].replace('064','06-4')
df['割当'] = df['割当'].replace('065','06-5')
df['割当'] = df['割当'].replace('066','06-6')
df['割当'] = df['割当'].replace('067','06-7')
df['割当'] = df['割当'].replace('083','08-3')
df['割当'] = df['割当'].replace('084','08-4')
df['割当'] = df['割当'].replace('105','10-5')
df['割当'] = df['割当'].replace('106','10-6')
df['割当'] = df['割当'].replace('107','10-7')
df['割当'] = df['割当'].replace('115','11-5')
df['割当'] = df['割当'].replace('116','11-6')

df['割当'] = df['割当'].replace('133','13-3')
df['割当'] = df['割当'].replace('138','13-8')
df['割当'] = df['割当'].replace('139','13-9')
df['割当'] = df['割当'].replace('1310','13-10')

df_comb_math_pret = df.copy()

In [222]:
df_comb_math_pret['割当'].unique()

array(['03-4', '01', '02', '03-6', '03-5', '-', '04-4', '04-7', '04-6',
       '04-8', '03', '04-5', '05', '06-4', '06-6', '10', '01-3', '09',
       '02-4', '07', '04', '02-5', '01-2', '01-1', '08-4', '06', '08-3',
       '11', '13-8', '12', 'KS', nan, '不参加', '06-7', '08', '13-3', '11-6',
       '13-9', '11-5', '13-10', 'パート分けナシ', '05-8', '10-5', '05-5', '05-7',
       '05-6', '10-6', '10-7', '13'], dtype=object)

In [223]:
df_math_setsumon_pretreated['割当'].unique()

array(['01', '02', '03-4', '03-5', '03-6', '03', '04-4', '04-5', '04-6',
       '04-7', '04-8', '05', '06-4', '06-5', '06-6', '11', '12', '13-3',
       '13-4', '13-5', '13-6', '13-7', '13-8', '13-9', '13-10', '06',
       '07', '08-3', '08-4', '09', '10', '01-1', '01-2', '01-3', '02-4',
       '02-5', '04', '06-7', '08', '11-5', '11-6', '05-5', '05-6', '05-7',
       '05-8', '10-5', '10-6', '10-7', '10-8', '05-9', 11, 12, '02-6',
       '04-9', '03-7', '09-5', '09-6', '09-7', '09-8', '06-1', '06-2',
       '06-3', '07-4', '07-5', '10-2', '10-3', '11-4', '11-7', '11-8',
       '11-9', '11-10', '01 ', '03-3', '12-2', '12-3', '03-8', '05-4'],
      dtype=object)

In [28]:
"""
2つのテーブルを結合させる
df1:採点者データの数学だけ抽出
df2:設問データの数学だけ抽出
"""
df1 = df_comb_math_pret
df2 = df_math_setsumon_pretreated
df_res = pandas_tool.merge(df1,df2, key = ["年度","採点回","学年","割当"],how = 'left')
df_res= df_res.drop_duplicates()
df_comb_math = df_res.copy()
df_res.head()

df1のキー重複
9399
df2のキー重複
8
df1のオブザベーション:9539
df2のオブザベーション:463
mergeのオブザベーション:9539


,スタッフコード,スタッフコード3,年度,採点回,科目コード,身分,性別,年齢,登録試験点数,大学,...,（1）配点,（2）配点,（3）配点,企画ペース,最終ペース_設問,模範解答の記述行数（１）,模範解答の記述行数（２）,模範解答の記述行数（３）,補足資料の枚数,分野
0,1060438022,3060438022,2021,夏,20,大学院生,男性,23.0,39.0,兵庫県立大学,...,6.0,7.0,12.0,140.0,99.3,9.0,10.0,38.0,12.0,二次関数
1,1060465851,3060465851,2021,夏,20,大学院生,男性,25.0,54.0,大阪市立大学,...,6.0,7.0,12.0,140.0,99.3,9.0,10.0,38.0,12.0,二次関数
2,1060530451,3060530451,2021,夏,20,大学院生,男性,24.0,69.0,名古屋工業大学,...,6.0,7.0,12.0,140.0,99.3,9.0,10.0,38.0,12.0,二次関数
3,1060612068,3060612068,2021,夏,20,大学生,男性,26.0,62.0,星薬科大学,...,6.0,7.0,12.0,140.0,99.3,9.0,10.0,38.0,12.0,二次関数
4,1060631414,3060631414,2021,夏,20,大学院生,男性,24.0,64.0,埼玉大学,...,6.0,7.0,12.0,140.0,99.3,9.0,10.0,38.0,12.0,二次関数


In [225]:
"""
Excelの書き出し 
"""

df = df_math_setsumon_pretreated

name = "採点者情報0720+0818_数学設問_結合前設問データ"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス

#Excel
with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name)) as writer:
    df.to_excel(writer, sheet_name='data1',encoding='utf-8-sig', index = False)

In [226]:
"""
Excelの書き出し 
"""

df = df_comb_math_pret

name = "採点者情報0720+0818_数学_結合前採点者データ"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス

#Excel
with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name)) as writer:
    df.to_excel(writer, sheet_name='data1',encoding='utf-8-sig', index = False)

In [29]:
"""
Excelの書き出し 
"""

df = df_comb_math

name = "採点者情報0720+0818_数学設問"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス

#csv
df.to_csv('{}/{}.csv'.format(path_folder,name),encoding='utf-8-sig',index=False)
#Excel
with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name)) as writer:
    df.to_excel(writer, sheet_name='data1',encoding='utf-8-sig', index = False)

In [215]:
#重複確認
df_res[df_res.index.duplicated()==True]

,スタッフコード,スタッフコード3,年度,採点回,科目コード_x,身分,性別,年齢,登録試験点数,大学,...,（2）配点,（3）配点,企画ペース,最終ペース_設問,模範解答の記述行数（１）,模範解答の記述行数（２）,模範解答の記述行数（３）,補足資料の枚数,科目コード_y,分野


In [33]:
"""
Excelの書き出し 集計用データマート
"""

name = "採点者情報0720+0818_pretreated_v2"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス


df1 = df_comb_jpn
df2 = df_comb_math
df3 = df_comb_eng

#csv
df1.to_csv('{}/{}.csv'.format(path_folder,"{}_{}".format(name,'jpn')),encoding='utf-8-sig',index=False)
df2.to_csv('{}/{}.csv'.format(path_folder,"{}_{}".format(name,'math')),encoding='utf-8-sig',index=False)
df3.to_csv('{}/{}.csv'.format(path_folder,"{}_{}".format(name,'eng')),encoding='utf-8-sig',index=False)

#Excel
with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name)) as writer:
    df1.to_excel(writer, sheet_name='国語',encoding='utf-8-sig', index = False)
    df2.to_excel(writer, sheet_name='数学',encoding='utf-8-sig', index = False)
    df3.to_excel(writer, sheet_name='英語',encoding='utf-8-sig', index = False)